In [1]:
from datetime import datetime, timedelta
import pandas as pd 
import datetime as dt
import fnmatch
import os
import pymysql
from sqlalchemy import create_engine
import sys
import json
import numpy as np
import warnings
from tqdm import tqdm
warnings.filterwarnings('ignore')

# Obtener la fecha de hoy
hoy = datetime.now()

# Calcular la fecha de ayer
ayer = hoy - timedelta(days=1)

# Formatear la fecha como "aaaa-mm-dd"
fecha_ayer_formato = ayer.strftime("%Y-%m-%d")

fecha_ayer_formato

'2023-12-25'

In [2]:
def encontrar_registros_adicionales(df1, df2, columnas_comparar):
    """
    Encuentra los registros adicionales en el segundo DataFrame en comparación con el primero.

    Parameters:
    - df1: Primer DataFrame
    - df2: Segundo DataFrame
    - columnas_comparar: Lista de columnas en las cuales comparar los DataFrames

    Returns:
    - DataFrame con los registros adicionales en el segundo DataFrame
    """
    # Realiza un merge con indicator=True
    merged = pd.merge(df1, df2, on=columnas_comparar, how='outer', indicator=True)

    # Filtra los resultados para obtener solo los registros en el segundo DataFrame
    adicionales_df2 = merged.loc[lambda x: x['_merge'] == 'right_only']

    # Elimina la columna _merge utilizada solo para la comparación
    adicionales_df2 = adicionales_df2.drop('_merge', axis=1)

    return adicionales_df2


In [3]:
cnx = pymysql.connect(host='159.89.90.197', port=3306, user='antonio_diaz', passwd="4nT0ni_0.dZ",
                           charset='utf8',db = 'volana')

In [4]:
path = f'./JERS/data/'
if not os.path.exists(path + fecha_ayer_formato): 
    os.makedirs(path+ fecha_ayer_formato) 

**Obtener fecha más actual**

In [5]:
from datetime import datetime

# Convierte las fechas a objetos datetime
fechas_dt = [datetime.strptime(fecha, '%Y-%m-%d') for fecha in os.listdir("./JERS/data/") if fecha != fecha_ayer_formato]

# Encuentra la fecha más reciente
ultima_fecha = max(fechas_dt)

ultima_fecha = ultima_fecha.strftime('%Y-%m-%d')

In [6]:
ultima_fecha

'2023-12-21'

# DEU

In [ ]:
cursor = cnx.cursor()
sql_query = f"""
            SELECT 
ids.id_distribuidor as `Clave Referencia`,
if(ids.tipo_prestamo = 'PRESTAMO PERSONAL RED', 'RED', cd.tipo_distribuidor) as 'tipo distribuidor',
"6" as `Tipo Operación`,
ids.id_red as `Contratante`,
IFNULL(dd.apellido_1,"X") as `Paterno Deudor`,
IFNULL(dd.apellido_2,"X") as `Materno Contratante`,
CONCAT(ifnull(dd.nombre_1,"")," ",ifnull(dd.nombre_2,"")) as `Nombre Contratante`,
"PF" as `Tipo de Persona`,
ifnull(dd.rfc,"X") as `RFC`,
CONCAT(ifnull(dd.calle_y_numero,"")," ",ifnull(dd.colonia,"")) as `Dirección`,
IFNULL(dd.delegacion,"X") as `Ciudad`,
IFNULL(dd.cp,"X") as `CP`,
IFNULL(dd.estado,"X") as `Estado`,
"N/A" as `Email`,
IFNULL(dd.telefono,"X") as `Teléfono`,
date(cd.fecha_aut_linea) as `Fecha Firma Contrato`,
if(cd.tipo_distribuidor = 'RED',ids.fin_credito,'2099-12-31') as `Fecha Vencimiento Contrato`,
if(cd.tipo_distribuidor = 'RED','16','0') as `Plazo`,
ids.id_red as `Crédito`,
"N" as `Anexo`,
cd.linea_de_credito  as `Monto Contrato`,
if(cd.tipo_distribuidor = 'RED','Semanal','Quincenal') as `Frecuencia de Pago`,
cd.proximo_pago_total_pago as `Pago Periódico`,
cd.linea_de_credito as `Monto Pagare`,
if(cd.tipo_distribuidor = 'RED',ids.tasa,"") AS `Tasa`,
dd.fecha_nacimiento as `Fecha de nacimiento contratante`,
IFNULL(dd.sexo,"X") as `Sexo`,
IFNULL(dd.curp,"X") as `CURP`
FROM cartera_distribuidor cd
LEFT JOIN (
		select 
			ca.id_distribuidor, 
            max(ca.id_red) as 'id_red', 
            max(ca.tipo_prestamo) as 'tipo_prestamo',
            max(tasa) as 'tasa',
            max(fin_credito) as 'fin_credito'
			from (
				select 
				if(tipo_prestamo = 'PRESTAMO PERSONAL RED', id_distribuidor_linea_de_credito, id_distribuidor) as 'id_distribuidor', 
				id_distribuidor as 'id_red',
				tipo_prestamo,
                tasa,
                fin_credito
				from colocacion_asociado
			) as ca 
		group by ca.id_distribuidor
) ids on cd.id_distribuidor = ids.id_distribuidor
LEFT JOIN demograficos_distribuidores dd on ids.id_distribuidor = dd.id
WHERE fecha = '{fecha_ayer_formato}' and 
ids.id_distribuidor IS NOT NULL and
dd.nombre_1 IS NOT NULL;
            """
cursor.execute(sql_query)
DEU = cursor.fetchall()

In [ ]:
DEU = pd.DataFrame(DEU,columns = ['Clave Referencia', 'tipo distribuidor', 'Tipo Operación',
       'Contratante', 'Paterno Deudor', 'Materno Contratante',
       'Nombre Contratante', 'Tipo de Persona', 'RFC', 'Dirección', 'Ciudad',
       'CP', 'Estado', 'Email', 'Teléfono', 'Fecha Firma Contrato',
       'Fecha Vencimiento Contrato', 'Plazo', 'Crédito', 'Anexo',
       'Monto Contrato', 'Frecuencia de Pago', 'Pago Periódico',
       'Monto Pagare', 'Tasa', 'Fecha de nacimiento contratante', 'Sexo',
       'CURP'])

## Adecuaciones

In [ ]:
DEU_id = np.array(DEU['Contratante'].unique(), dtype = int)
DEU['Fecha Firma Contrato'] = pd.to_datetime(DEU['Fecha Firma Contrato'])
print(DEU.shape)
for name in DEU.select_dtypes(include=['object']):
    DEU[name] = DEU[name].str.replace("|", " ")
print("Hay que revisar DEU" if (DEU.isna().any(0).sum() > 2) else "Todo bien con DEU")
DEU.head(1)

In [ ]:
DEU.to_csv(f'{path}/{fecha_ayer_formato}/DEU_{fecha_ayer_formato}.txt', sep ='|', index = False, header = False)

In [241]:
DEU_ant = pd.read_csv(f"{path}{ultima_fecha}/DEU_{ultima_fecha}.txt", sep = '|', header = None)
DEU_ant.columns = DEU.columns
DEU_ant.shape[0], DEU.shape[0]

(10909, 10909)

In [242]:
DEU = pd.read_csv(f"{path}{fecha_ayer_formato}/DEU_{fecha_ayer_formato}.txt", sep = '|', header = None)
DEU.columns = DEU_ant.columns

In [243]:
DEU.Email = DEU.Email.fillna("")
DEU_ant.Email = DEU_ant.Email.fillna("")

In [244]:
DEU_new = encontrar_registros_adicionales(DEU, DEU_ant, list(DEU.columns))

DEU_new.shape

(73, 28)

In [245]:
DEU_new.to_csv(f'{path}/{fecha_ayer_formato}/DEU_{fecha_ayer_formato}_diferencial.txt', sep ='|', index = False, header = False)

# MOVA

In [246]:
cursor = cnx.cursor()
sql_query = f"""
select 
    id_distribuidor as `Contrato`,
    "N" as `Anexo`,
    -- if(tipo_distribuidor = 'RED', id_distribuidor_linea_de_credito, id_distribuidor) as `Número de documento`,
    id_distribuidor as `Número de documento`,
    date(if(tipo_distribuidor = 'RED',fin_credito,'2099-12-31')) as `Fecha Vencimiento`,
    ROUND(sum(capital), 2) as `Monto Capital`,
    ROUND(sum(interes), 2) as `Monto intérés`,
    ROUND(sum(iva), 2) as `Monto IVA`,
    "FAC" AS `Tipo de Movimiento`
FROM
    (SELECT 
    cd.id_distribuidor,
    "N" as `Anexo`,
    cd.tipo_distribuidor,
    ca.capital,
    ca.interes,
    ca.iva,
    cd.fecha,
    ca.fin_credito,
    ca.no_pagos,
    ca.id_distribuidor_linea_de_credito
    FROM cartera_distribuidor cd
    LEFT JOIN colocacion_asociado ca on cd.id_distribuidor = ca.id_distribuidor
    WHERE fecha = '{fecha_ayer_formato}' and ca.tipo_prestamo = 'PRESTAMO PERSONAL RED') as temp

GROUP BY temp.id_distribuidor, fecha, fin_credito, tipo_distribuidor-- , temp.id_distribuidor_linea_de_credito
            """
cursor.execute(sql_query)
MOVA = cursor.fetchall()

In [247]:
MOVA = pd.DataFrame(MOVA, columns = ['Contrato', 'Anexo', 'Número de documento', 'Fecha Vencimiento',
       'Monto Capital', 'Monto intérés', 'Monto IVA', 'Tipo de Movimiento'])

## Adecuaciones

In [248]:
MOVA['Fecha Vencimiento'] = MOVA['Fecha Vencimiento'].fillna("2099-12-31")
print("Hay que revisar MOVA" if (MOVA.isna().any(0).sum() > 0) else "Todo bien con MOVA")
MOVA['Contrato'] = MOVA['Contrato'].astype(int)
MOVA_id = MOVA['Contrato']
print(MOVA.shape)
MOVA.head(1)

Todo bien con MOVA
(3696, 8)


,Contrato,Anexo,Número de documento,Fecha Vencimiento,Monto Capital,Monto intérés,Monto IVA,Tipo de Movimiento
0,81210,N,81210,2021-10-25,34000.0,8245.12,1319.22,FAC


In [249]:
MOVA = MOVA.loc[[mi in DEU_id for mi in MOVA_id]]
MOVA = MOVA.drop_duplicates('Contrato')
np.setdiff1d(MOVA['Contrato'], DEU_id)

array([], dtype=int32)

In [250]:
new_mova = []
for row in tqdm(MOVA.iterrows()):
    #new_mova = pd.concat([new_mova, row[1]], axis = 1)
    #new_mova.append(row[1])
    num = 16
    for i in range(1, num + 1):
        new_row = row[1].copy()
        new_row['Número de documento'] = str(new_row['Número de documento']) + "_"+str(i)
        new_row['Monto Capital'] = new_row['Monto Capital'] / num
        new_row['Monto intérés'] = new_row['Monto intérés'] / num
        new_row['Monto IVA'] = new_row['Monto IVA'] / num
        #new_mova = pd.concat([new_mova, new_row], axis = 1)
        new_mova.append(new_row)

2292it [00:02, 843.18it/s] 


In [251]:
MOVA = pd.DataFrame(new_mova)
MOVA.head()

,Contrato,Anexo,Número de documento,Fecha Vencimiento,Monto Capital,Monto intérés,Monto IVA,Tipo de Movimiento
0,81210,N,81210_1,2021-10-25,2125.0,515.32,82.45125,FAC
0,81210,N,81210_2,2021-10-25,2125.0,515.32,82.45125,FAC
0,81210,N,81210_3,2021-10-25,2125.0,515.32,82.45125,FAC
0,81210,N,81210_4,2021-10-25,2125.0,515.32,82.45125,FAC
0,81210,N,81210_5,2021-10-25,2125.0,515.32,82.45125,FAC


In [252]:
MOVA.to_csv(f'{path}/{fecha_ayer_formato}/MOVA_{fecha_ayer_formato}.txt', sep ='|', index = False, header = False)

In [253]:
MOVA_ant = pd.read_csv(f"{path}{ultima_fecha}/MOVA_{ultima_fecha}.txt", sep = '|', header = None)
MOVA_ant = MOVA_ant.infer_objects()

MOVA_ant.columns = MOVA.columns
MOVA_ant.shape[0], MOVA.shape[0]

(36624, 36672)

In [254]:
MOVA = pd.read_csv(f"{path}{fecha_ayer_formato}/MOVA_{fecha_ayer_formato}.txt", sep = '|', header = None)
MOVA.columns = MOVA_ant.columns

In [255]:
MOVA_new = encontrar_registros_adicionales(MOVA, MOVA_ant, list(MOVA.columns))

MOVA_new.to_csv(f'{path}/{fecha_ayer_formato}/MOVA_{fecha_ayer_formato}_diferencial.txt', sep ='|', index = False, header = False)

In [256]:
MOVA_ant.query("Contrato == 163779")

,Contrato,Anexo,Número de documento,Fecha Vencimiento,Monto Capital,Monto intérés,Monto IVA,Tipo de Movimiento


In [257]:
MOVA_new

,Contrato,Anexo,Número de documento,Fecha Vencimiento,Monto Capital,Monto intérés,Monto IVA,Tipo de Movimiento
36672,152089,N,152089_1,2023-09-05,3624.99875,810.3400,129.66125,FAC
36673,152089,N,152089_2,2023-09-05,3624.99875,810.3400,129.66125,FAC
36674,152089,N,152089_3,2023-09-05,3624.99875,810.3400,129.66125,FAC
36675,152089,N,152089_4,2023-09-05,3624.99875,810.3400,129.66125,FAC
36676,152089,N,152089_5,2023-09-05,3624.99875,810.3400,129.66125,FAC
...,...,...,...,...,...,...,...,...
36731,164904,N,164904_12,2024-01-11,7499.99750,1250.0025,200.00000,FAC
36732,164904,N,164904_13,2024-01-11,7499.99750,1250.0025,200.00000,FAC
36733,164904,N,164904_14,2024-01-11,7499.99750,1250.0025,200.00000,FAC
36734,164904,N,164904_15,2024-01-11,7499.99750,1250.0025,200.00000,FAC


# MOVIC

In [258]:
cursor = cnx.cursor()
sql_query = f"""
SELECT 
coalesce(cd.id_red, pd.id_distribuidor) as `Contrato`,
"N" AS `Anexo`,
-- coalesce(if(cd.tipo_prestamo = 'PRESTAMO PERSONAL RED', cd.id_red, cd.id_distribuidor), pd.id_distribuidor) as `Número de documento`,
coalesce(cd.id_distribuidor, pd.id_distribuidor) as `Número de documento`,
DATE(pd.fecha_pago) as `Fecha Depósito`,
DATE(pd.fecha_aplicacion_pago) as `Fecha Aplicación`,
IF(pd.seguro_asociado>0,-pd.seguro_asociado,pd.seguro_asociado) as `Monto de Seguro Asociado`,
IF(pd.seguro_distribuidor>0,-pd.seguro_distribuidor,pd.seguro_distribuidor) as `Monto de Seguro Distribuidor`,
"1" AS `Tipo de Movimiento`,
pd.procedencia as `Cuenta`,
pd.id_pago as `paymentId`

FROM pagos_desglosados pd

left JOIN (
		select 
			ca.id_distribuidor, max(ca.id_red) as 'id_red', max(ca.tipo_prestamo) as 'tipo_prestamo'
			from (
				select 
				if(tipo_prestamo = 'PRESTAMO PERSONAL RED', id_distribuidor_linea_de_credito, id_distribuidor) as 'id_distribuidor', 
				id_distribuidor as 'id_red',
				tipo_prestamo
				from colocacion_asociado
			) as ca 
		group by ca.id_distribuidor
) cd on pd.id_distribuidor = cd.id_red
WHERE date(pd.fecha_pago) >='2021-01-01'
"""
cursor.execute(sql_query)
MOVIC = cursor.fetchall()

In [259]:
MOVIC = pd.DataFrame(MOVIC, columns = ['Contrato', 'Anexo', 'Número de documento', 'Fecha Depósito',
       'Fecha Aplicación', 'Monto de Seguro Asociado',
       'Monto de Seguro Distribuidor', 'Tipo de Movimiento', 'Cuenta',
       'paymentId'])

In [260]:
print("Hay que revisar MOVIC" if (MOVIC.isna().any(0).sum() > 0) else "Todo bien con MOVIC")
MOVIC.Contrato = MOVIC.Contrato.astype(int) 

MOVIC_id = MOVIC['Contrato']
print(MOVIC.shape)
MOVIC.head(1)

Todo bien con MOVIC
(299826, 10)


,Contrato,Anexo,Número de documento,Fecha Depósito,Fecha Aplicación,Monto de Seguro Asociado,Monto de Seguro Distribuidor,Tipo de Movimiento,Cuenta,paymentId
0,80782,N,80782,2021-10-04,2021-10-04,-50,-25,1,BANCOMER,105234


In [261]:
sum([mi in DEU_id for mi in MOVIC_id])

246300

In [262]:
MOVIC = MOVIC.loc[[mi in DEU_id for mi in MOVIC_id]]

In [263]:
np.setdiff1d(MOVIC['Contrato'], DEU_id)

array([], dtype=int32)

In [264]:
MOVIC = MOVIC.drop_duplicates(subset = ["paymentId"])

In [265]:
MOVIC.to_csv(f'{path}/{fecha_ayer_formato}/MOVIC_{fecha_ayer_formato}.txt', sep ='|', index = False, header = False)

In [266]:
MOVIC_ant = pd.read_csv(f"{path}{ultima_fecha}/MOVIC_{ultima_fecha}.txt", sep = '|', header = None)
MOVIC_ant = MOVIC_ant.infer_objects()

MOVIC_ant.columns = MOVIC.columns
MOVIC_ant.shape[0], MOVIC.shape[0]

(246484, 126898)

In [267]:
MOVIC = pd.read_csv(f"{path}{fecha_ayer_formato}/MOVIC_{fecha_ayer_formato}.txt", sep = '|', header = None)
MOVIC.columns = MOVIC_ant.columns

In [268]:
MOVIC_new = encontrar_registros_adicionales(MOVIC, MOVIC_ant, list(MOVIC.columns))

MOVIC_new = MOVIC_new.drop_duplicates(subset = ["paymentId"])

MOVIC_new.to_csv(f'{path}/{fecha_ayer_formato}/MOVIC_{fecha_ayer_formato}_diferencial.txt', sep ='|', index = False, header = False)

# MOVO

In [269]:
cursor = cnx.cursor()
sql_query = f"""
select
coalesce(cd.id_red, pd.id_distribuidor) as `Contrato`,
"N" AS `Anexo`,
-- coalesce(if(cd.tipo_prestamo = 'PRESTAMO PERSONAL RED', cd.id_red, cd.id_distribuidor), pd.id_distribuidor) as `Número de documento`,
coalesce(cd.id_distribuidor, pd.id_distribuidor) as `Número de documento`,
DATE(pd.fecha_pago) as `Fecha Depósito`,
DATE(pd.fecha_aplicacion_pago) as `Fecha Aplicación`,
IF(pd.capital>0,-pd.capital,pd.capital) as `Monto de Principal`,
IF(pd.interes>0,-pd.interes,pd.interes) as `Monto de interés`,
IF(pd.iva>0,-pd.iva,pd.iva) as `Monto de IVA`,
"PAG" AS `Tipo de Movimiento`,
pd.procedencia as `Cuenta`,
pd.id_pago as `paymentId`
FROM pagos_desglosados pd
left JOIN (
		select 
			ca.id_distribuidor, max(ca.id_red) as 'id_red', max(ca.tipo_prestamo) as 'tipo_prestamo'
			from (
				select 
				if(tipo_prestamo = 'PRESTAMO PERSONAL RED', id_distribuidor_linea_de_credito, id_distribuidor) as 'id_distribuidor', 
				id_distribuidor as 'id_red',
				tipo_prestamo
				from colocacion_asociado
			) as ca 
		group by ca.id_distribuidor
) cd on pd.id_distribuidor = cd.id_red
where date(fecha_pago) >= '2021-01-01'
            """
cursor.execute(sql_query)
MOVO = cursor.fetchall()

In [270]:
MOVO = pd.DataFrame(MOVO, columns =  ['Contrato', 'Anexo', 'Número de documento', 'Fecha Depósito',
       'Fecha Aplicación', 'Monto de Principal', 'Monto de interés',
       'Monto de IVA', 'Tipo de Movimiento', 'Cuenta', 'paymentId'])

In [271]:
print("Hay que revisar MOVO" if (MOVO.isna().any(0).sum() > 0) else "Todo bien con MOVO")
MOVO['Contrato'] = MOVO['Contrato'].astype(int)
MOVO_id = MOVO['Contrato']
print(MOVO.shape)
MOVO.head(1)

Todo bien con MOVO
(299826, 11)


,Contrato,Anexo,Número de documento,Fecha Depósito,Fecha Aplicación,Monto de Principal,Monto de interés,Monto de IVA,Tipo de Movimiento,Cuenta,paymentId
0,80782,N,80782,2021-10-04,2021-10-04,-1199.75,-103.78,-16.6,PAG,BANCOMER,105234


In [272]:
sum([mi in DEU_id for mi in MOVO_id])

246300

In [273]:
MOVO = MOVO.loc[[mi in DEU_id for mi in MOVO_id]]

In [274]:
np.setdiff1d(MOVO['Contrato'], DEU_id)

array([], dtype=int32)

In [275]:
MOVO = MOVO.drop_duplicates(subset = ["paymentId"])

In [276]:
MOVO.to_csv(f'./{path}/{fecha_ayer_formato}/MOVO_{fecha_ayer_formato}.txt', sep ='|', index = False, header = False)

In [277]:
MOVO_ant = pd.read_csv(f"{path}{ultima_fecha}/MOVO_{ultima_fecha}.txt", sep = '|', header = None)
MOVO_ant = MOVO_ant.infer_objects()

MOVO_ant.columns = MOVO.columns
MOVO_ant.shape[0], MOVO.shape[0]

(246484, 126898)

In [278]:
MOVO = pd.read_csv(f"{path}{fecha_ayer_formato}/MOVO_{fecha_ayer_formato}.txt", sep = '|', header = None)
MOVO.columns = MOVO_ant.columns

In [279]:
MOVO_new = encontrar_registros_adicionales(MOVO, MOVO_ant, list(MOVO.columns))

MOVO_new = MOVO_new.drop_duplicates(subset = ["paymentId"])

MOVO_new.to_csv(f'{path}/{fecha_ayer_formato}/MOVO_{fecha_ayer_formato}_diferencial.txt', sep ='|', index = False, header = False)

In [280]:
cnx.close()

# Enviando al SFTP

In [281]:
import paramiko

In [49]:
# Configura la conexión SFTP
def enviar(name):
    hostname = 'linqsy.dyndns.org'
    port = 22
    username = 'Volana'
    password = '*v0lana-202311'

    # Crea una instancia de la clase Transport
    transport = paramiko.Transport((hostname, port))

    # Conéctate con el usuario y contraseña
    transport.connect(username=username, password=password)


    # Ruta local y remota del archivo
    local_path = f'./JERS/data/{fecha_ayer_formato}/{name}_{fecha_ayer_formato}_diferencial.txt'
    #remote_path = f'/archivos_{fecha_ayer_formato}/{name}_{fecha_ayer_formato}.txt'
    #
    remote_path = f'/{name}{fecha_ayer_formato.replace("-","")}.txt'
    # Crea una instancia de la clase SFTP
    sftp = paramiko.SFTPClient.from_transport(transport)

    # Sube el archivo
    sftp.put(local_path, remote_path)

    # Cierra la conexión SFTP
    sftp.close()

    # Cierra la conexión de transporte
    transport.close()

In [63]:
for name in ['DEU', 'MOVA', 'MOVIC', 'MOVO']:
    enviar(name)
    print(f"{name} enviado correctamente")

SSHException: Unable to connect to linqsy.dyndns.org: [WinError 10060] Se produjo un error durante el intento de conexión ya que la parte conectada no respondió adecuadamente tras un periodo de tiempo, o bien se produjo un error en la conexión establecida ya que el host conectado no ha podido responder

In [114]:
DEU.query("Contratante == 170116")

,Clave Referencia,tipo distribuidor,Tipo Operación,Contratante,Paterno Deudor,Materno Contratante,Nombre Contratante,Tipo de Persona,RFC,Dirección,...,Crédito,Anexo,Monto Contrato,Frecuencia de Pago,Pago Periódico,Monto Pagare,Tasa,Fecha de nacimiento contratante,Sexo,CURP
6024,150267,RED,6,170116,ROSAS,ISIDORO,GUADALUPE,PF,ROIG871212,LUIS CORDOBA REYES no ext.MZ 13 LT 20 Wencesl...,...,170116,N,15000,Semanal,0.0,15000,142.0,1987-12-12 00:00:00,FEMENINO,ROIG871212MPLSSD04
6045,150338,RED,6,170116,CHAVERO,RESENDIZ,MARIA DE LA LUZ,PF,CARL790912,12 DE DICIEMBRE no ext.MZ 32 LT 32 Wenceslao ...,...,170116,N,15000,Semanal,0.0,15000,142.0,1979-09-12 00:00:00,FEMENINO,CARL790912MDFHSZ03
6064,150414,RED,6,170116,SANTIAGO,GONZALEZ,AZUCENA,PF,SAGA881204,LUIS CORDOBA REYES no ext.MZ11 LT4 Wenceslao ...,...,170116,N,15000,Semanal,0.0,15000,142.0,1988-12-04 00:00:00,FEMENINO,SAGA881204MMCNNZ09
6066,150417,RED,6,170116,SANTIAGO,GONZALEZ,DANIELA ALEXANDRA,PF,SAGD040808,LUIS CORDOBA REYES no ext.MZ11 LT4 Wenceslao ...,...,170116,N,15000,Semanal,0.0,15000,147.0,2004-08-08 00:00:00,FEMENINO,SAGD040808MMCNNNA7
6101,150770,RED,6,170116,CHAVERO,RESENDIZ,CATALINA,PF,CARC560330,12 DE DICIEMBRE no ext.MZ32 LT32 Wenceslao Vi...,...,170116,N,15000,Semanal,0.0,15000,142.0,1956-03-30 00:00:00,FEMENINO,CARC560330MTLHST02


In [133]:
f = pd.read_csv("./JERS/data/2023-11-30/DEU_2023-11-30.txt", sep = '|')
sum(f.iloc[:, 3] == 170116)

5

In [135]:
f = pd.read_csv("./JERS/data/2023-11-30/DEU_2023-11-30_diferencial.txt", sep = '|')
sum(f.iloc[:, 3] == 170116)

0

In [136]:
f

,100087,RED,6,151294,CID,LOPEZ,GUADALUPE,PF,CILG720829,CALLE 6 no ext.609 Melesio Portillo,...,151294.1,N,2000,Semanal,0.0,2000.1,85.0,1972-08-29,FEMENINO,CILG720829MVZDPD05
0,100094,RED,6,151294,BALDERAS,PALMA,MARIA ISABEL,PF,BAPI840708,C 2 A NTE no ext.lt 5 CENTRO,...,151294,N,2000,Semanal,0.0,2000,120.000000,1984-07-08,FEMENINO,BAPI840708MVZLLS01
1,100474,RED,6,151294,MORENO,FLORES,JULIA,PF,MOFJ650216,AND A TEJEDA no ext.1 M 67 San Jose,...,151294,N,2000,Semanal,0.0,2000,115.000000,1965-02-16,FEMENINO,MOFJ650216MVZRLL06
2,100546,RED,6,151294,NEGRETE,LOPEZ,PATRICIA,PF,NELP731120,AV 8 PTE no ext.SN Melesio Portillo,...,151294,N,2000,Semanal,0.0,2000,130.000000,1973-11-20,FEMENINO,NELP731120MVZGPT02
3,102779,RED,6,160645,VENEGAS,CHAVEZ,NOEMI,PF,VECN811225,CALLE HIPODROMO no ext.286 Santa Paula,...,160645,N,2000,Semanal,0.0,2000,115.000000,1981-12-25,FEMENINO,VECN811225MJCNHM05
4,105401,RED,6,166064,CEDILLO,CASTILLO,MA CONSEPCION,PF,CECC720505,CLL 5 DE MAYO no ext.10 San Pablo,...,166064,N,19000,Semanal,1820.0,19000,178.639999,1972-05-05,FEMENINO,X
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,96699,CLUB,6,96699,SOSA,RICARDEZ,CARLOTA BEATRIZ,PF,SORC750115,AND PERIFERICO no ext.218 MZ 18 Deportiva Inf...,...,96699,N,56000,Quincenal,7139.0,56000,NaN,1975-01-15,FEMENINO,SORC750115MTCSCR01
172,97062,RED,6,97062,LARA,RAMIREZ,ANGELA,PF,LARA731003,RIA LOS NARANJO 1RA no ext.S N Los Naranjos ...,...,97062,N,17000,Semanal,925.0,17000,144.460007,1973-10-03,FEMENINO,LARA731003MTCRMN03
173,97777,RED,6,98763,GONZALEZ,COLORADO,NANCY,PF,GOCN810912,CALLE 12 ET AV 6 Y 8 PROG no ext.sn Huatusco...,...,98763,N,2000,Semanal,0.0,2000,135.000000,1981-09-12,FEMENINO,GOCN810912MVZNLN04
174,98276,RED,6,98763,MARTINEZ,MARTINEZ,GLORIA AURORA,PF,MAMG760729,PRADERAS DE AXOL no ext.MZA 1 937 B Los Manan...,...,98763,N,19000,Semanal,4200.0,19000,210.470001,1976-07-29,FEMENINO,MAMG760729MVZRRL07


In [167]:
ant = pd.read_csv("./JERS/data/2023-11-29/DEU_2023-11-29.txt", sep = '|', header=None)
ant.columns = DEU.columns
sum(ant.iloc[:, 3] == 170116)

0

In [168]:
act = pd.read_csv("./JERS/data/2023-11-30/DEU_2023-11-30.txt", sep = '|', header=None)
act.columns = DEU.columns
sum(act.iloc[:, 3] == 170116)

5

In [169]:
np.setdiff1d(act['Contratante'], ant['Contratante'])

array([170094, 170116, 170707, 170730, 170778, 170789, 170798, 170864,
       170887, 170901, 170923, 170931, 171051, 171098, 171112, 171131,
       171138, 171140, 171142, 171145, 171152, 171173, 171180, 171191,
       171193], dtype=int64)

In [179]:
act.query("Contratante == 168607")

,Clave Referencia,tipo distribuidor,Tipo Operación,Contratante,Paterno Deudor,Materno Contratante,Nombre Contratante,Tipo de Persona,RFC,Dirección,...,Crédito,Anexo,Monto Contrato,Frecuencia de Pago,Pago Periódico,Monto Pagare,Tasa,Fecha de nacimiento contratante,Sexo,CURP
5886,149212,RED,6,168607,VELASCO,RODRIGUEZ,SANDRA SARAHI,PF,VERS901019,CALLE 20 no ext.14 Estado de Mexico VMC,...,168607,N,19000,Semanal,3025.0,19000,142.000000,1990-10-19,FEMENINO,VERS901019MMCLDN06
5888,149216,RED,6,168607,ZARAGOZA,GUTIERREZ,MARGARITA,PF,ZAGM670911,CALLE 19 no ext.140 3 Estado de Mexico VMC,...,168607,N,10000,Semanal,0.0,10000,142.000000,1967-09-11,FEMENINO,ZAGM670911MDFRTR02
5890,149218,RED,6,168607,VELASCO,RODRIGUEZ,ALEJANDRA,PF,VERA850325,CALLE 19 no ext.139 Estado de Mexico VMC,...,168607,N,24000,Semanal,3925.0,24000,178.270004,1985-03-25,FEMENINO,VERA850325MMCLDL01
5897,149288,RED,6,168607,SERRANO,SANCHEZ,BRENDA GUADALUPE,PF,SESB950626,LAGO TRASIMENO no ext.190 A AGUA AZUL,...,168607,N,10000,Semanal,1385.0,10000,161.000000,1995-06-26,FEMENINO,SESB950626MDFRNR07
5903,149334,RED,6,168607,MARTINEZ,CHAVEZ,ARGELIA,PF,MACA830520,LAGO TRASIMENO no ext.198 AGUA AZUL,...,168607,N,10000,Semanal,0.0,10000,147.000000,1983-05-20,FEMENINO,MACA830520MDFRHR07
5907,149358,RED,6,168607,SANCHEZ,GONZALEZ,ISELDA GUADALUPE,PF,SAGI720315,LAGO TRASIMENO no ext.190 A AGUA AZUL,...,168607,N,10000,Semanal,0.0,10000,147.000000,1972-03-15,FEMENINO,SAGI720315MMCNNS04


In [175]:
np.sort(news['Contratante'].unique())

array([   116,    797,   2564,   3481,   4913,   5728,   5846,   8082,
         8514,   8633,   8994,  11364,  18377,  18392,  19005,  21372,
        35175,  36018,  39115,  41923,  41954,  42541,  45700,  49058,
        49194,  53936,  58634,  59982,  65871,  71670,  72585,  72671,
        72755,  74111,  74349,  74420,  75222,  75542,  76453,  76890,
        77025,  80127,  81580,  91125,  91630,  93200,  93832,  94364,
        96699,  97062,  98015,  98763,  99692, 105594, 115814, 124276,
       135406, 143017, 149805, 150088, 150610, 151193, 151294, 151537,
       153292, 154410, 154813, 154950, 157331, 157729, 159099, 160152,
       160363, 160645, 160842, 161455, 162174, 162240, 162275, 162304,
       162341, 162565, 162649, 162785, 163201, 163656, 164143, 164381,
       164868, 165113, 165211, 165581, 165683, 165716, 166064, 166250,
       166500, 166649, 166991, 167702, 168607, 168894, 169004, 169280,
       169388, 169932], dtype=int64)

In [172]:
#sum(
news = encontrar_registros_adicionales(act, ant, list(DEU.columns))#['Contratante'] == 170116)

In [153]:
act.query("Contratante == 170116")

,Clave Referencia,tipo distribuidor,Tipo Operación,Contratante,Paterno Deudor,Materno Contratante,Nombre Contratante,Tipo de Persona,RFC,Dirección,...,Crédito,Anexo,Monto Contrato,Frecuencia de Pago,Pago Periódico,Monto Pagare,Tasa,Fecha de nacimiento contratante,Sexo,CURP
6024,150267,RED,6,170116,ROSAS,ISIDORO,GUADALUPE,PF,ROIG871212,LUIS CORDOBA REYES no ext.MZ 13 LT 20 Wencesl...,...,170116,N,15000,Semanal,0.0,15000,142.0,1987-12-12 00:00:00,FEMENINO,ROIG871212MPLSSD04
6045,150338,RED,6,170116,CHAVERO,RESENDIZ,MARIA DE LA LUZ,PF,CARL790912,12 DE DICIEMBRE no ext.MZ 32 LT 32 Wenceslao ...,...,170116,N,15000,Semanal,0.0,15000,142.0,1979-09-12 00:00:00,FEMENINO,CARL790912MDFHSZ03
6064,150414,RED,6,170116,SANTIAGO,GONZALEZ,AZUCENA,PF,SAGA881204,LUIS CORDOBA REYES no ext.MZ11 LT4 Wenceslao ...,...,170116,N,15000,Semanal,0.0,15000,142.0,1988-12-04 00:00:00,FEMENINO,SAGA881204MMCNNZ09
6066,150417,RED,6,170116,SANTIAGO,GONZALEZ,DANIELA ALEXANDRA,PF,SAGD040808,LUIS CORDOBA REYES no ext.MZ11 LT4 Wenceslao ...,...,170116,N,15000,Semanal,0.0,15000,147.0,2004-08-08 00:00:00,FEMENINO,SAGD040808MMCNNNA7
6101,150770,RED,6,170116,CHAVERO,RESENDIZ,CATALINA,PF,CARC560330,12 DE DICIEMBRE no ext.MZ32 LT32 Wenceslao Vi...,...,170116,N,15000,Semanal,0.0,15000,142.0,1956-03-30 00:00:00,FEMENINO,CARC560330MTLHST02


In [154]:
ant.query("Contratante == 170116")

,Clave Referencia,tipo distribuidor,Tipo Operación,Contratante,Paterno Deudor,Materno Contratante,Nombre Contratante,Tipo de Persona,RFC,Dirección,...,Crédito,Anexo,Monto Contrato,Frecuencia de Pago,Pago Periódico,Monto Pagare,Tasa,Fecha de nacimiento contratante,Sexo,CURP


In [180]:
ant.shape

(10909, 28)

In [181]:
act.shape

(10909, 28)